<a href="https://colab.research.google.com/github/wellBorges/PUC_PLN/blob/main/03_a_representa%C3%A7%C3%A3o_textual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalação dos pacotes necessários
!pip install nltk==3.8.1
!pip install unidecode==1.3.8
!pip install scikit-learn==1.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.5 MB/s eta 0:00:00


In [2]:
# Importações da biblioteca padrão
import datetime
import os
import re
import string
import sys
import warnings

# Importações de bibliotecas de terceiros
import nltk
import pandas as pd

from nltk.corpus import stopwords

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from unidecode import unidecode

# Downloads do NLTK
nltk.download('punkt')
nltk.download("stopwords")

# Configurações e comandos específicos (por exemplo, desativar avisos)
warnings.filterwarnings('ignore')

print("Pacotes importados com sucesso; notebook pronto para uso!")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Pacotes importados com sucesso; notebook pronto para uso!


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# declara as funções utilitárias do notebook
def formata_msg(nivel, msg):
  """
    Formata uma mensagem de log incluindo o nível de severidade, timestamp
    e a mensagem.

    Parâmetros:
    - nivel (str): Nível de severidade da mensagem (ex: 'INFO', 'ERROR', 'WARNING').
    - msg (str): A mensagem de log propriamente dita.

    Retorna:
    - str: A mensagem de log formatada.
  """

  timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

  return f"[{nivel}] {timestamp} - {msg}"



print(formata_msg("INFO", "Funções utilitárias prontas para utilização."))
print(formata_msg("INFO", f"Versão do Python: {sys.version} "))

[INFO] 2024-06-19 11:09:54 - Funções utilitárias prontas para utilização.
[INFO] 2024-06-19 11:09:54 - Versão do Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0] 


# Definição do Corpus

In [10]:
documentos =\
[
  # No Meio do Caminho - Carlos Drumond de Andrade
  "No meio do caminho tinha uma pedra\n"\
  "tinha uma pedra no meio do caminho\n"\
  "tinha uma pedra\n"\
  "no meio do caminho tinha uma pedra.\n"\
  "Nunca me esquecerei desse acontecimento\n"\
  "na vida de minhas retinas tão fatigadas.\n"\
  "Nunca me esquecerei que no meio do caminho\n"\
  "tinha uma pedra\n"\
  "tinha uma pedra no meio do caminho\n"\
  "no meio do caminho tinha uma pedra."
  ,
  # Quadrilha - Carlos Drumond de Andrade
  "João amava Teresa que amava Raimundo\n"\
  "que amava Maria que amava Joaquim que amava Lili,\n"\
  "que não amava ninguém.\n"\
  "João foi para os Estados Unidos, Teresa para o convento,\n"\
  "Raimundo morreu de desastre, Maria ficou para tia,\n"\
  "Joaquim suicidou-se e Lili casou com J. Pinto Fernandes\n"\
  "que não tinha entrado na história. pedra" #palavra pedra incluída pelo aluno para identificar similaridade
 ]

In [11]:
print(formata_msg("INFO", f"Documentos:\n{documentos}"))

[INFO] 2024-06-19 11:17:54 - Documentos:
['No meio do caminho tinha uma pedra\ntinha uma pedra no meio do caminho\ntinha uma pedra\nno meio do caminho tinha uma pedra.\nNunca me esquecerei desse acontecimento\nna vida de minhas retinas tão fatigadas.\nNunca me esquecerei que no meio do caminho\ntinha uma pedra\ntinha uma pedra no meio do caminho\nno meio do caminho tinha uma pedra.', 'João amava Teresa que amava Raimundo\nque amava Maria que amava Joaquim que amava Lili,\nque não amava ninguém.\nJoão foi para os Estados Unidos, Teresa para o convento,\nRaimundo morreu de desastre, Maria ficou para tia,\nJoaquim suicidou-se e Lili casou com J. Pinto Fernandes\nque não tinha entrado na história. pedra']


## Preprocessamento

<b> Atividade </b>

1) Escreva uma método que realiza o pré-processamento da lista de <b>documentos</b>.

O método deve, para cada documento:
- tokenizar cada palavra
- remover stopwords
- remover números
- remover pontuções
- remover acentos

In [12]:
def pre_processa_texto(texto):
    """
    Preprocessa o texto fornecido realizando várias etapas de limpeza.

    Etapas:
    1. Tokeniza o texto.
    2. Converte os tokens para minúsculos.
    3. Remove stopwords em português.
    4. Remove números dos tokens.
    5. Exclui tokens que são pontuações.
    6. Remove acentuações dos tokens.

    Parâmetros:
    texto (str): O texto a ser preprocessado.

    Retorna:
    list: Lista de tokens preprocessados.
    """

    # Tokeniza o texto usando um padrão para capturar palavras e pontuações.
    padrao = r"\w+(?:'\w+)?|[^\w\s]"
    tokens_preprocessados = re.findall(padrao, texto)

    # Converte os tokens para minúsculos para padronizar a capitalização.
    tokens_preprocessados = [token.lower() for token in tokens_preprocessados]

    # Remove stopwords para reduzir o conjunto de tokens a palavras significativas.
    portugues_stops = stopwords.words('portuguese')
    tokens_preprocessados = [token for token in tokens_preprocessados
                             if token not in portugues_stops]

    # Remove números, pois geralmente não contribuem para o significado do texto.
    tokens_preprocessados = [re.sub(r'\d+', '', token) for token in tokens_preprocessados
                             if re.sub(r'\d+', '', token)]

    # Exclui tokens que são pontuações, pois raramente são úteis para análise de texto.
    tokens_preprocessados = [token for token in tokens_preprocessados
                             if token not in string.punctuation]

    # Remove acentuações para padronizar os tokens.
    tokens_preprocessados = [unidecode(token) for token in tokens_preprocessados]

    return ' '.join(tokens_preprocessados)


In [13]:
documentos_preprocessados =  [pre_processa_texto(poema)  for poema in documentos]
print(formata_msg("INFO", f"Documentos preprocessados:\n{documentos_preprocessados}"))

[INFO] 2024-06-19 11:18:00 - Documentos preprocessados:
['meio caminho pedra pedra meio caminho pedra meio caminho pedra nunca esquecerei desse acontecimento vida retinas tao fatigadas nunca esquecerei meio caminho pedra pedra meio caminho meio caminho pedra', 'joao amava teresa amava raimundo amava maria amava joaquim amava lili amava ninguem joao estados unidos teresa convento raimundo morreu desastre maria ficou tia joaquim suicidou lili casou j pinto fernandes entrado historia pedra']


# Representação Textual

## Bag of Words

Faça o CountVectorizer nos documentos da variável <b>documentos</b> considerando binary = True

In [14]:
def aplica_bow(documentos_preprocessados):
    """
    Analisa documentos usando BoW e retorna o vocabulário extraído e um
    DataFrame contendo os valores BoW para cada termo em cada documento.

    Parâmetros:
    - documentos_preprocessados: Lista de documentos preprocessados (strings).

    Retorna:
    - Um dicionário contendo o vocabulário extraído e o DataFrame dos valores
      BoW.
    """
    # Inicializa o CountVectorizer
    vectorizer = CountVectorizer(binary=True)

    # Transforma os documentos em uma matriz binária
    bow_matrix = vectorizer.fit_transform(documentos_preprocessados)

    # Obtém o vocabulário extraído
    vocabulario = vectorizer.get_feature_names_out()

    # Cria um DataFrame para os valores bow.
    df = pd.DataFrame(
        bow_matrix.T.todense(),
        index=vocabulario,
        columns=[f"doc{i + 1}" for i in range(bow_matrix.shape[0])]
    )

    return {"vocabulario": vocabulario, "bow_dataframe": df}

In [15]:
bow_dict = aplica_bow(documentos_preprocessados)
print(formata_msg("INFO", f"Vocabulário BoW:\n{bow_dict['vocabulario']}"))
print(formata_msg("INFO", f"Dataframe BoW:\n{bow_dict['bow_dataframe']}"))

[INFO] 2024-06-19 11:18:05 - Vocabulário BoW:
['acontecimento' 'amava' 'caminho' 'casou' 'convento' 'desastre' 'desse'
 'entrado' 'esquecerei' 'estados' 'fatigadas' 'fernandes' 'ficou'
 'historia' 'joao' 'joaquim' 'lili' 'maria' 'meio' 'morreu' 'ninguem'
 'nunca' 'pedra' 'pinto' 'raimundo' 'retinas' 'suicidou' 'tao' 'teresa'
 'tia' 'unidos' 'vida']
[INFO] 2024-06-19 11:18:05 - Dataframe BoW:
               doc1  doc2
acontecimento     1     0
amava             0     1
caminho           1     0
casou             0     1
convento          0     1
desastre          0     1
desse             1     0
entrado           0     1
esquecerei        1     0
estados           0     1
fatigadas         1     0
fernandes         0     1
ficou             0     1
historia          0     1
joao              0     1
joaquim           0     1
lili              0     1
maria             0     1
meio              1     0
morreu            0     1
ninguem           0     1
nunca             1     0
pedra  

## TD-IDF

1) Faça o TDIFTVectorizer nos documentos da variável <b>documentos</b> sem alterar nenhum parâmetro.

In [18]:
def aplica_tfidf(documentos_preprocessados):
    """
    Analisa documentos usando TF-IDF e retorna o vocabulário extraído e um
    DataFrame contendo os valores TF-IDF para cada termo em cada documento.

    Parâmetros:
    - documentos_preprocessados: Lista de documentos preprocessados (strings).

    Retorna:
    - Um dicionário contendo o vocabulário extraído e o DataFrame dos valores
      TF-IDF.
    """
    # Inicializa o TfidfVectorizer
    vectorizer = TfidfVectorizer()

    # Transforma os documentos em uma matriz TF-IDF
    tfidf_matrix = vectorizer.fit_transform(documentos_preprocessados)

    # Obtém o vocabulário extraído
    vocabulario = vectorizer.get_feature_names_out()

    # Cria um DataFrame para os valores TF-IDF
    df = pd.DataFrame(
        tfidf_matrix.T.todense(),
        index=vocabulario,
        columns=[f"doc{i + 1}" for i in range(tfidf_matrix.shape[0])]
    )

    return {"vocabulario": vocabulario, "tfidf_dataframe": df}


In [19]:
tfidf_dict = aplica_tfidf(documentos_preprocessados)
print(formata_msg("INFO", f"Vocabulário TF-IDF:\n{tfidf_dict['vocabulario']}"))
print(formata_msg("INFO", f"Dataframe TF-IDF:\n{tfidf_dict['tfidf_dataframe']}"))

[INFO] 2024-06-19 11:19:47 - Vocabulário TF-IDF:
['acontecimento' 'amava' 'caminho' 'casou' 'convento' 'desastre' 'desse'
 'entrado' 'esquecerei' 'estados' 'fatigadas' 'fernandes' 'ficou'
 'historia' 'joao' 'joaquim' 'lili' 'maria' 'meio' 'morreu' 'ninguem'
 'nunca' 'pedra' 'pinto' 'raimundo' 'retinas' 'suicidou' 'tao' 'teresa'
 'tia' 'unidos' 'vida']
[INFO] 2024-06-19 11:19:47 - Dataframe TF-IDF:
                   doc1      doc2
acontecimento  0.094999  0.000000
amava          0.000000  0.695112
caminho        0.569993  0.000000
casou          0.000000  0.115852
convento       0.000000  0.115852
desastre       0.000000  0.115852
desse          0.094999  0.000000
entrado        0.000000  0.115852
esquecerei     0.189998  0.000000
estados        0.000000  0.115852
fatigadas      0.094999  0.000000
fernandes      0.000000  0.115852
ficou          0.000000  0.115852
historia       0.000000  0.115852
joao           0.000000  0.231704
joaquim        0.000000  0.231704
lili           0.0000